In [142]:
import pandas as pd 
import numpy as np
import openpyxl as xl

In [150]:
df = pd.read_csv('Results of COE Bidding Exercise.csv', usecols=range(8))
# Filter for only cat A vehicles 
df = df[df.Category == 'Cat A (Cars up to 1600cc and 97kW)']
df = df[df.Year <= 2022]
df = df[['Bidding Exercise', 'Announcement Date', 'Year', 'Quota', 'Quota Premium', 'Total Bids Received', 'Number of Successful Bids']]
df.head()

,Bidding Exercise,Announcement Date,Year,Quota,Quota Premium,Total Bids Received,Number of Successful Bids
130,December 2022 Second Open Bidding Exercise,21/12/2022,2022,473,88007,618,472
135,December 2022 First Open Bidding Exercise,07/12/2022,2022,458,88503,616,454
140,November 2022 Second Open Bidding Exercise,23/11/2022,2022,459,90589,622,444
145,November 2022 First Open Bidding Exercise,09/11/2022,2022,460,87235,687,460
150,October 2022 Second Open Bidding Exercise,19/10/2022,2022,559,81089,714,558


In [151]:
# Format first col to groupby
def format(text):
    words = ['Open ', 'First ', 'Second ']
    for word in words:
        text = text.replace(word, '')
    return text.strip()

df['Bidding Exercise'] = df['Bidding Exercise'].astype(str).apply(format)
df.head()

,Bidding Exercise,Announcement Date,Year,Quota,Quota Premium,Total Bids Received,Number of Successful Bids
130,December 2022 Bidding Exercise,21/12/2022,2022,473,88007,618,472
135,December 2022 Bidding Exercise,07/12/2022,2022,458,88503,616,454
140,November 2022 Bidding Exercise,23/11/2022,2022,459,90589,622,444
145,November 2022 Bidding Exercise,09/11/2022,2022,460,87235,687,460
150,October 2022 Bidding Exercise,19/10/2022,2022,559,81089,714,558


In [152]:
# Calculate average quota premium for each month across the two bidding periods
cols = ['Quota', 'Quota Premium', 'Total Bids Received', 'Number of Successful Bids']
for col in cols:
    df[col] = df[col].astype(int)
    
df = df.groupby(['Bidding Exercise', 'Year'], sort=False, as_index=False)
df = df[cols].mean()
df.head()

,Bidding Exercise,Year,Quota,Quota Premium,Total Bids Received,Number of Successful Bids
0,December 2022 Bidding Exercise,2022,465.5,88255.0,617.0,463.0
1,November 2022 Bidding Exercise,2022,459.5,88912.0,654.5,452.0
2,October 2022 Bidding Exercise,2022,561.5,80795.0,719.0,559.5
3,September 2022 Bidding Exercise,2022,549.5,85000.0,741.0,531.0
4,August 2022 Bidding Exercise,2022,552.5,83944.5,762.0,546.0


In [153]:
# Column displaying previous average premium (the month before)
df['Previous'] = df['Quota Premium'].shift(-1)
df = df.dropna()
df.head()

,Bidding Exercise,Year,Quota,Quota Premium,Total Bids Received,Number of Successful Bids,Previous
0,December 2022 Bidding Exercise,2022,465.5,88255.0,617.0,463.0,88912.0
1,November 2022 Bidding Exercise,2022,459.5,88912.0,654.5,452.0,80795.0
2,October 2022 Bidding Exercise,2022,561.5,80795.0,719.0,559.5,85000.0
3,September 2022 Bidding Exercise,2022,549.5,85000.0,741.0,531.0,83944.5
4,August 2022 Bidding Exercise,2022,552.5,83944.5,762.0,546.0,78450.0


In [155]:
# Create size of labour force data
labourforce_df = pd.read_csv('labourforce.csv')
years = labourforce_df.iloc[8, 1:]
size = labourforce_df.iloc[9, 1:]

labourforce_df = pd.DataFrame({
    'Year': years.values,
    'Total Labour Force (Thousand)': size.values
})

labourforce_df = labourforce_df.replace('na', np.nan).dropna()
labourforce_df['Year'] = labourforce_df['Year'].astype(int)
labourforce_df = labourforce_df[labourforce_df.Year >= 2002]
labourforce_df.head()

,Year,Total Labour Force (Thousand)
1,2022,3754.0
2,2021,3608.0
3,2020,3714.0
4,2019,3743.0
5,2018,3676.0


In [158]:
# Include size of labour force data
merged_df = pd.merge(df, labourforce_df, on='Year', how='left')
merged_df.to_csv('/Users/germaine/hacknroll/output.csv')

In [160]:
data = pd.read_csv('output.csv')